# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [22]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [14]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df['prestige'], df['admit'])

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [23]:
df = df.join(pd.get_dummies(df['prestige'], prefix = 'prestige'))
df.head()

,admit,gre,gpa,prestige,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,3.0,0,0,1,0
1,1,660.0,3.67,3.0,0,0,1,0
2,1,800.0,4.00,1.0,1,0,0,0
3,1,640.0,3.19,4.0,0,0,0,1
4,0,520.0,2.93,4.0,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: Three

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [25]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix = 'prestige')
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1.0':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


In [26]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc['prestige_1.0'], handCalc['admit'])

admit,0,1
prestige_1.0,,
0,243,93
1,28,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [36]:
odds_prestige_1 = 33 * 1.0 / 28
print odds_prestige_1

1.17857142857


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [37]:
odds_prestige_not1 = 93 * 1.0 / 243
print odds_prestige_not1

0.382716049383


#### 3.3 Calculate the odds ratio

In [38]:
print odds_prestige_1 / odds_prestige_not1

3.07949308756


#### 3.4 Write this finding in a sentenance: 

Answer: People who attended #1 ranked colleges were more likely to be admitted. 

#### 3.5 Print the cross tab for prestige_4

In [39]:
pd.crosstab(handCalc['prestige_4.0'], handCalc['admit'])

admit,0,1
prestige_4.0,,
0,216,114
1,55,12


#### 3.6 Calculate the OR 

In [40]:
print (12 / 55.0) / (114 / 216.0)

0.413397129187


#### 3.7 Write this finding in a sentence

Answer: People who attended #4 ranked colleges were less likely to be admitted. 

## Part 4. Analysis

In [41]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [42]:
# manually add the intercept
data['intercept'] = 1.0

In [48]:
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,0,380.0,3.61,0,1,0,1.0
1,1,660.0,3.67,0,1,0,1.0
2,1,800.0,4.00,0,0,0,1.0
3,1,640.0,3.19,0,0,1,1.0
4,0,520.0,2.93,0,0,1,1.0


#### 4.1 Set the covariates to a variable called train_cols

In [49]:
train_cols = data.columns[1:]
train_cols

Index([u'gre', u'gpa', u'prestige_2.0', u'prestige_3.0', u'prestige_4.0',
       u'intercept'],
      dtype='object')

#### 4.2 Fit the model

In [50]:
model = sm.Logit(data['admit'], data[train_cols])
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [51]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Thu, 23 Feb 2017   Pseudo R-squ.:                 0.08166
Time:                        10:14:34   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [58]:
# odds ratios
OR = np.exp(result.params)
print OR

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [66]:
# confidence intervaks
print np.exp(result.conf_int())

                     0         1
gre           1.000074  1.004372
gpa           1.136120  4.183113
prestige_2.0  0.272168  0.942767
prestige_3.0  0.133377  0.515419
prestige_4.0  0.093329  0.479411
intercept     0.002207  0.194440


#### 4.5 Interpret the OR of Prestige_2

Answer: Students who went to a #2 ranked college were less likely to be admitted.

#### 4.6 Interpret the OR of GPA

Answer: Students with higher GPA were more likely to be admitted.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [88]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [103]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [96]:
print combos.head()

       0         1    2    3
0  220.0  2.260000  1.0  1.0
1  220.0  2.260000  2.0  1.0
2  220.0  2.260000  3.0  1.0
3  220.0  2.260000  4.0  1.0
4  220.0  2.453333  1.0  1.0


#### 5.1 Recreate the dummy variables

In [104]:
# recreate the dummy variables
# keep only what we need for making predictions

combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
combos.head()

,gre,gpa,prestige,intercept
0,220.0,2.260000,1.0,1.0
1,220.0,2.260000,2.0,1.0
2,220.0,2.260000,3.0,1.0
3,220.0,2.260000,4.0,1.0
4,220.0,2.453333,1.0,1.0


In [105]:
combos = combos.join(pd.get_dummies(combos['prestige'], prefix = 'prestige').ix[:,1:])
del combos['prestige']

In [106]:
combos.head()

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0
0,220.0,2.260000,1.0,0,0,0
1,220.0,2.260000,1.0,1,0,0
2,220.0,2.260000,1.0,0,1,0
3,220.0,2.260000,1.0,0,0,1
4,220.0,2.453333,1.0,0,0,0


#### 5.2 Make predictions on the enumerated dataset

In [126]:
combos['prob'] = result.predict(combos)

In [127]:
combos.tail()

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0,prob
395,800.0,3.806667,1.0,0,0,1,0.545972
396,800.0,4.000000,1.0,0,0,0,0.985399
397,800.0,4.000000,1.0,1,0,0,0.946509
398,800.0,4.000000,1.0,0,1,0,0.934534
399,800.0,4.000000,1.0,0,0,1,0.582995


#### 5.3 Interpret findings for the last 4 observations

Answer: Going to a #4 ranked school has the biggest marginal impact on a student's likelihood of being admitted.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.